In [1]:
# ===================================================================================
# 1. CONFIGURAÇÃO DO AMBIENTE E IMPORTAÇÕES
# ===================================================================================
!pip install splink -q

import duckdb
import pandas as pd
from splink import Linker, SettingsCreator, DuckDBAPI
import splink.comparison_library as cl
from splink import block_on
from splink.exploratory import completeness_chart, profile_columns
from splink.blocking_analysis import cumulative_comparisons_to_be_scored_from_blocking_rules_chart, n_largest_blocks

print("Ambiente configurado com sucesso.")

Ambiente configurado com sucesso.


In [2]:
# ===================================================================================
# 2. CONEXÃO, PRÉ-PROCESSAMENTO E AMOSTRAGEM (VERSÃO ULTRA-LEVE)
# ===================================================================================
path_dados = 'dados_catalogo/processed/dados_consolidados_filtrados.parquet'
con = duckdb.connect(database=':memory:', read_only=False)
db_api = DuckDBAPI(connection=con)

sql_completo = f"""
CREATE OR REPLACE VIEW dados_completos AS
SELECT *,
    list_distinct(list_sort(string_to_array(trim(BB_Cast), ','))) as BB_Cast_array,
    list_distinct(list_sort(string_to_array(trim(BB_Directors), ','))) as BB_Directors_array
FROM read_parquet('{path_dados}');
"""
con.execute(sql_completo)

# CORREÇÃO PRINCIPAL: Reduzimos drasticamente o tamanho da amostra para o guia didático.
sql_amostra = "CREATE OR REPLACE VIEW dados_amostra AS SELECT * FROM dados_completos USING SAMPLE 30000 ROWS;"
con.execute(sql_amostra)
print("View 'dados_amostra' (30 mil linhas) criada para uma exploração muito mais rápida.")

View 'dados_amostra' (30 mil linhas) criada para uma exploração muito mais rápida.


In [3]:
# ===================================================================================
# 3. ANÁLISE EXPLORATÓRIA (NA AMOSTRA)
# ===================================================================================
settings_preliminares = SettingsCreator(
    link_type="dedupe_only",
    unique_id_column_name="BB_UID" # O Linker agora "sabe" qual é a coluna de ID
)

linker_para_analise = Linker("dados_amostra", settings_preliminares, db_api=db_api)
print("Linker preliminar para análise criado com sucesso.")

Linker preliminar para análise criado com sucesso.


In [4]:
# ===================================================================================
# 4. ANÁLISE DE REGRAS DE BLOQUEIO (VERSÃO FINALÍSSIMA E CORRETA)
# ===================================================================================
# O objetivo é usar as ferramentas do Splink na AMOSTRA para diagnosticar e escolher
# regras de bloqueio eficientes.

# --- Passo 4.1: Diagnóstico de Regras de Coluna Única ---
# Usamos o linker_para_analise, que já foi criado na célula 3,
# para executar queries SQL de diagnóstico de forma segura.

print("--- Diagnóstico: Maiores Blocos para 'BB_Year' ---")
sql_diag_year = """
SELECT "BB_Year", COUNT(*) AS contagem_no_bloco
FROM dados_amostra
GROUP BY "BB_Year"
ORDER BY contagem_no_bloco DESC
LIMIT 5;
"""
display(linker_para_analise.misc.query_sql(sql_diag_year, output_type="pandas"))


print("\n--- Diagnóstico: Maiores Blocos para 'BB_Title' ---")
sql_diag_title = """
SELECT "BB_Title", COUNT(*) AS contagem_no_bloco
FROM dados_amostra
GROUP BY "BB_Title"
ORDER BY contagem_no_bloco DESC
LIMIT 5;
"""
display(linker_para_analise.misc.query_sql(sql_diag_title, output_type="pandas"))


# --- Passo 4.2: Avaliar um conjunto de regras de bloqueio inteligentes ---
# Com base no diagnóstico, focaremos apenas em regras combinadas.
regras_de_bloqueio_candidatas = [
    block_on("BB_Title", "BB_Year"),
    block_on("substr(BB_Title, 1, 4)", "BB_Year"),
    block_on("substr(BB_Directors, 1, 6)", "BB_Year"),
    block_on("substr(BB_Cast, 1, 6)", "BB_Year"),
    block_on("BB_Title", "substr(BB_Directors, 1, 6)"),
]

print("\n--- Análise Cumulativa de Comparações para Regras Candidatas ---")
# CORREÇÃO FINAL: Voltamos a usar a função standalone, que é o correto.
# Ela NÃO é um método do linker. Passamos todos os parâmetros necessários,
# como db_api e unique_id_column_name, para que ela tenha o contexto para rodar.
cumulative_comparisons_to_be_scored_from_blocking_rules_chart(
    table_or_tables="dados_amostra",
    blocking_rules=regras_de_bloqueio_candidatas,
    db_api=db_api,
    link_type="dedupe_only",
    unique_id_column_name="BB_UID"
)

--- Diagnóstico: Maiores Blocos para 'BB_Year' ---


,BB_Year,contagem_no_bloco
0,NaN,5773
1,2019.0,1877
2,2021.0,1777
3,2022.0,1650
4,2020.0,1587



--- Diagnóstico: Maiores Blocos para 'BB_Title' ---


,BB_Title,contagem_no_bloco
0,None,674
1,Carmen,6
2,The Mummy,6
3,Star Trek: Nemesis,6
4,The Island,6



--- Análise Cumulativa de Comparações para Regras Candidatas ---


...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

alt.Chart(...)

In [5]:
# ===================================================================================
# 5. CONFIGURAÇÃO DO MODELO (COM BLOQUEIO SIMPLIFICADO)
# ===================================================================================
# CORREÇÃO: Usamos um conjunto de regras mais simples e estrito para garantir que o 'predict' seja rápido.
regras_de_bloqueio_para_guia = [
    block_on("substr(BB_Title, 1, 4)", "BB_Year"),
    block_on("substr(BB_Directors, 1, 8)", "BB_Year"),
]

settings_para_exploracao = SettingsCreator(
    link_type="dedupe_only",
    unique_id_column_name="BB_UID",
    blocking_rules_to_generate_predictions=regras_de_bloqueio_para_guia,
    comparisons=[
        cl.JaroWinklerAtThresholds("BB_Title", [0.9, 0.7]),
        cl.ExactMatch("BB_Year"),
        cl.ArrayIntersectAtSizes("BB_Directors_array", [1]),
        cl.ArrayIntersectAtSizes("BB_Cast_array", [3, 1]),
    ],
    retain_matching_columns=True,
    retain_intermediate_calculation_columns=True
)

print("Configuração (settings) para exploração simplificada criada com sucesso.")

Configuração (settings) para exploração simplificada criada com sucesso.


In [6]:
# ===================================================================================
# 6. TREINAMENTO DO MODELO (NA AMOSTRA)
# ===================================================================================
# CORREÇÃO FINAL: Em vez de tentar modificar o linker de análise, criamos um NOVO
# Linker para o modelo, usando as configurações finais que acabamos de definir.
# Este linker ainda aponta para a AMOSTRA, para manter o processo rápido e exploratório.

print("--- Criando o Linker de modelo para a amostra ---")
linker_modelo = Linker("dados_amostra", settings_para_exploracao, db_api=db_api)
print("Linker de modelo criado com sucesso.")


# --- 6.1: Estimando Lambda ---
print("\n--- Estimando Lambda (Probabilidade de Match Aleatório) ---")
regras_deterministicas = [
    block_on("BB_Title", "BB_Year", "BB_Directors"),
    block_on("BB_Title", "BB_Year", "BB_Cast")
]
linker_modelo.training.estimate_probability_two_random_records_match(regras_deterministicas, recall=0.7)


# --- 6.2: Estimando as Probabilidades 'u' ---
print("\n--- Estimando probabilidades 'u' ---")
linker_modelo.training.estimate_u_using_random_sampling(max_pairs=1e6)


# --- 6.3: Estimando as Probabilidades 'm' (Expectation Maximisation) ---
print("\n--- Iniciando sessões de treinamento EM ---")
linker_modelo.training.estimate_parameters_using_expectation_maximisation(block_on("BB_Title", "BB_Year"))
linker_modelo.training.estimate_parameters_using_expectation_maximisation(block_on("BB_Directors", "BB_Year"))
print("\nTreinamento do modelo concluído!")

--- Criando o Linker de modelo para a amostra ---
Linker de modelo criado com sucesso.

--- Estimando Lambda (Probabilidade de Match Aleatório) ---


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Probability two random records match is estimated to be  3.17e-08.
This means that amongst all possible pairwise record comparisons, one in 31,498,950.00 are expected to match.  With 449,985,000 total possible comparisons, we expect a total of around 14.29 matching pairs
You are using the default value for `max_pairs`, which may be too small and thus lead to inaccurate estimates for your model's u-parameters. Consider increasing to 1e8 or 1e9, which will result in more accurate estimates, but with a longer run time.
----- Estimating u probabilities using random sampling -----


...


Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - BB_Title (no m values are trained).
    - BB_Year (no m values are trained).
    - BB_Directors_array (no m values are trained).
    - BB_Cast_array (no m values are trained).



--- Iniciando sessões de treinamento EM ---


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(l."BB_Title" = r."BB_Title") AND (l."BB_Year" = r."BB_Year")

Parameter estimates will be made for the following comparison(s):
    - BB_Directors_array
    - BB_Cast_array

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - BB_Title
    - BB_Year

Iteration 1: Largest change in params was 0.205 in probability_two_random_records_match
Iteration 2: Largest change in params was 0.0504 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.00603 in probability_two_random_records_match
Iteration 4: Largest change in params was 0.000722 in probability_two_random_records_match
Iteration 5: Largest change in params was 8.65e-05 in probability_two_random_records_match

EM converged after 5 iterations

Your model is not yet fully trained. Missing estimates for:
    - BB_Title (no m values are traine


Treinamento do modelo concluído!


In [7]:
# ===================================================================================
# 7. VITRINE DE FERRAMENTAS DE ANÁLISE E VISUALIZAÇÃO (NA AMOSTRA)
# ===================================================================================
# Todas as chamadas agora usam o 'linker_modelo' que foi devidamente treinado.

print("--- Gráfico de Pesos de Correspondência (Match Weights) ---")
display(linker_modelo.visualisations.match_weights_chart())

print("\n--- Executando predição na amostra ---")
df_predict_amostra = linker_modelo.inference.predict()

print("\n--- Gerando o Dashboard 'Comparison Viewer' ---")
linker_modelo.visualisations.comparison_viewer_dashboard(df_predict_amostra, "comparison_viewer.html", overwrite=True)
print("Dashboard salvo em 'comparison_viewer.html'.")

# ... (e assim por diante para todas as outras funções de visualização e clusterização)

# ===================================================================================
# 8. CONCLUSÃO E CAMINHO PARA A PRODUÇÃO
# ===================================================================================
caminho_modelo = "modelo_vod_exploratorio_v1.json"
linker_modelo.misc.save_model_to_json(caminho_modelo, overwrite=True)
print(f"Modelo treinado na amostra foi salvo em: {caminho_modelo}")

--- Gráfico de Pesos de Correspondência (Match Weights) ---


alt.VConcatChart(...)


--- Executando predição na amostra ---


Blocking time: 0.08 seconds
Predict time: 1.01 seconds

 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'BB_Year':
    m values not fully trained



--- Gerando o Dashboard 'Comparison Viewer' ---
Dashboard salvo em 'comparison_viewer.html'.
Modelo treinado na amostra foi salvo em: modelo_vod_exploratorio_v1.json
